# Shooting Method Notebook
Boundary-value solver by shooting with multiple IVP integrators (RK4, forward Euler, backward Euler) and root finders (Newton, bisection, secant). The intent is to keep every step explicit and inspectable.

Run the cells from top to bottom. Edit only the **Problem definition** cell to specify your ODE and boundary data. The remaining cells build the integrators, assemble the shooting residual, apply the chosen root finder, and plot trajectories, residuals, and per-iteration timings.

In [ ]:
import time
from typing import Callable, Dict, List, Tuple

import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')

## Problem definition (edit this cell)
- Provide `ode_rhs(x, y, yp)` so that `y'' = ode_rhs(x, y, y')`.
- Set the interval `[a, b]` and boundary values `y(a) = alpha`, `y(b) = beta`.
- Pick the root-finding method (Newton / bisection / secant) and IVP integrator (RK4 / Euler / backward Euler).
- `initial_guess` is the starting slope guess. `secondary_guess` supplies a second slope for secant; `bracket` supplies a sign-changing pair for bisection.
- `true_solution` is optional for plotting; set to `None` to disable.

In [ ]:
# === User input block ===
# Everything below is safe to edit. Keep other cells untouched unless you know what you are doing.

def ode_rhs(x: float, y: float, yp: float) -> float:
    'Second derivative: y'' = f(x, y, y'
    return -np.pi**2 * y  # Example: y'' + pi^2 y = 0 -> y = A sin(pi x) + B cos(pi x)

a, b = 0.0, 1.0               # interval endpoints
alpha, beta = 0.0, 1.0        # boundary values y(a), y(b)

initial_guess = 1.0           # slope guess y'(a) for Newton/secant
secondary_guess = 0.0         # second slope guess for secant (ignored otherwise)
bracket = (-5.0, 5.0)         # slope bracket for bisection (residuals must differ in sign)

step_size = 0.01              # positive step; smaller -> more accurate, slower
integrator = 'rk4'            # options: 'rk4', 'euler', 'backward_euler'

root_method = 'secant'        # options: 'newton', 'bisection', 'secant'
residual_tol = 1e-8           # stop when |residual| <= residual_tol
max_iters = 40                # safety cap on root-finder iterations

newton_fd_step = 1e-3         # finite-difference step for Newton's slope derivative
slope_tol = 1e-10             # stop if successive slopes change less than this (secant/Newton)

backward_euler_tol = 1e-10    # Newton solve tolerance inside backward Euler step
backward_euler_maxiter = 12   # Newton iterations for each backward Euler step

def true_solution(x: np.ndarray) -> np.ndarray:
    'Exact solution for comparison; return None to disable plotting it.'
    return None

## Helpers: integration and residual evaluation
This cell defines IVP integrators (RK4, forward Euler, backward Euler), numerical partials needed for backward Euler, and the shooting residual.

In [ ]:
def numerical_partials(f: Callable[[float, float, float], float], x: float, y: float, yp: float, eps: float = 1e-6) -> Tuple[float, float]:
    'Central finite differences for df/dy and df/d(yp).'
    fy = (f(x, y + eps, yp) - f(x, y - eps, yp)) / (2 * eps)
    fyp = (f(x, y, yp + eps) - f(x, y, yp - eps)) / (2 * eps)
    return fy, fyp


def euler_step(f: Callable[[float, float, float], float], x: float, y: float, yp: float, h: float) -> Tuple[float, float, float]:
    fval = f(x, y, yp)
    return x + h, y + h * yp, yp + h * fval


def rk4_step(f: Callable[[float, float, float], float], x: float, y: float, yp: float, h: float) -> Tuple[float, float, float]:
    k1_y = yp
    k1_yp = f(x, y, yp)

    k2_y = yp + 0.5 * h * k1_yp
    k2_yp = f(x + 0.5 * h, y + 0.5 * h * k1_y, yp + 0.5 * h * k1_yp)

    k3_y = yp + 0.5 * h * k2_yp
    k3_yp = f(x + 0.5 * h, y + 0.5 * h * k2_y, yp + 0.5 * h * k2_yp)

    k4_y = yp + h * k3_yp
    k4_yp = f(x + h, y + h * k3_y, yp + h * k3_yp)

    y_next = y + (h / 6.0) * (k1_y + 2 * k2_y + 2 * k3_y + k4_y)
    yp_next = yp + (h / 6.0) * (k1_yp + 2 * k2_yp + 2 * k3_yp + k4_yp)
    return x + h, y_next, yp_next


def backward_euler_step(
    f: Callable[[float, float, float], float],
    x: float,
    y: float,
    yp: float,
    h: float,
    tol: float,
    maxiter: int,
) -> Tuple[float, float, float]:
    'Implicit step solved with Newton on (y_{n+1}, y'_{n+1}).'
    x1 = x + h
    y_guess = y + h * yp
    yp_guess = yp + h * f(x, y, yp)

    for _ in range(maxiter):
        f_val = f(x1, y_guess, yp_guess)
        F1 = y_guess - y - h * yp_guess
        F2 = yp_guess - yp - h * f_val

        fy, fyp = numerical_partials(f, x1, y_guess, yp_guess)
        J11, J12 = 1.0, -h
        J21, J22 = -h * fy, 1.0 - h * fyp
        det = J11 * J22 - J12 * J21
        if abs(det) < 1e-14:
            raise RuntimeError('Backward Euler Jacobian is nearly singular')
        delta_y = (-F1 * J22 - (-F2) * J12) / det
        delta_yp = (J11 * -F2 - J21 * -F1) / det

        y_guess += delta_y
        yp_guess += delta_yp

        if max(abs(delta_y), abs(delta_yp)) < tol:
            return x1, y_guess, yp_guess

    raise RuntimeError('Backward Euler Newton solver did not converge')


def integrate_ivp(
    f: Callable[[float, float, float], float],
    a_: float,
    b_: float,
    y0: float,
    yp0: float,
    h: float,
    method: str,
    be_tol: float,
    be_maxiter: int,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    'Integrate y'' = f with chosen method; returns xs, ys, yps.'
    xs = [a_]
    ys = [y0]
    yps = [yp0]
    x = a_
    y = y0
    yp_curr = yp0

    while x < b_:
        step = min(h, b_ - x)
        if method == 'euler':
            x, y, yp_curr = euler_step(f, x, y, yp_curr, step)
        elif method == 'rk4':
            x, y, yp_curr = rk4_step(f, x, y, yp_curr, step)
        elif method == 'backward_euler':
            x, y, yp_curr = backward_euler_step(f, x, y, yp_curr, step, be_tol, be_maxiter)
        else:
            raise ValueError(f"Unknown integrator '{method}'")
        xs.append(x)
        ys.append(y)
        yps.append(yp_curr)

    return np.array(xs), np.array(ys), np.array(yps)


def shooting_residual(slope: float, integrator_name: str, h: float) -> Tuple[float, Tuple[np.ndarray, np.ndarray, np.ndarray]]:
    xs, ys, yps = integrate_ivp(
        ode_rhs,
        a,
        b,
        alpha,
        slope,
        h,
        integrator_name,
        backward_euler_tol,
        backward_euler_maxiter,
    )
    return ys[-1] - beta, (xs, ys, yps)

## Root finders with bookkeeping
Each method records slope, residual, per-iteration wall time, and the trajectory produced by that slope. Newton uses a centered finite-difference for the slope derivative.

In [ ]:
def residual_derivative_fd(slope: float, delta: float, integrator_name: str, h: float) -> float:
    f_plus, _ = shooting_residual(slope + delta, integrator_name, h)
    f_minus, _ = shooting_residual(slope - delta, integrator_name, h)
    return (f_plus - f_minus) / (2 * delta)


def run_newton(s0: float, tol: float, maxiter: int, delta: float, h: float, integrator_name: str) -> List[Dict]:
    history: List[Dict] = []
    slope = s0
    for _ in range(maxiter):
        t0 = time.perf_counter()
        res, traj = shooting_residual(slope, integrator_name, h)
        dt = time.perf_counter() - t0
        history.append({"slope": slope, "residual": res, "time": dt, "traj": traj})
        if abs(res) <= tol:
            break

        deriv = residual_derivative_fd(slope, delta, integrator_name, h)
        if abs(deriv) < 1e-14:
            raise RuntimeError('Newton derivative is ~0; choose a different slope or step')

        new_slope = slope - res / deriv
        if abs(new_slope - slope) < slope_tol:
            slope = new_slope
            break
        slope = new_slope
    return history


def run_secant(s0: float, s1: float, tol: float, maxiter: int, h: float, integrator_name: str) -> List[Dict]:
    history: List[Dict] = []
    t0 = time.perf_counter()
    r0, traj0 = shooting_residual(s0, integrator_name, h)
    history.append({"slope": s0, "residual": r0, "time": time.perf_counter() - t0, "traj": traj0})

    t0 = time.perf_counter()
    r1, traj1 = shooting_residual(s1, integrator_name, h)
    history.append({"slope": s1, "residual": r1, "time": time.perf_counter() - t0, "traj": traj1})

    for _ in range(maxiter - 2):
        if abs(r1 - r0) < 1e-14:
            raise RuntimeError('Secant denominator is ~0; adjust guesses')
        s2 = s1 - r1 * (s1 - s0) / (r1 - r0)

        t0 = time.perf_counter()
        r2, traj2 = shooting_residual(s2, integrator_name, h)
        dt = time.perf_counter() - t0
        history.append({"slope": s2, "residual": r2, "time": dt, "traj": traj2})

        if abs(r2) <= tol or abs(s2 - s1) < slope_tol:
            break
        s0, r0 = s1, r1
        s1, r1 = s2, r2

    return history


def run_bisection(br: Tuple[float, float], tol: float, maxiter: int, h: float, integrator_name: str) -> List[Dict]:
    left, right = br
    t0 = time.perf_counter()
    r_left, traj_left = shooting_residual(left, integrator_name, h)
    history: List[Dict] = [{"slope": left, "residual": r_left, "time": time.perf_counter() - t0, "traj": traj_left}]

    t0 = time.perf_counter()
    r_right, traj_right = shooting_residual(right, integrator_name, h)
    history.append({"slope": right, "residual": r_right, "time": time.perf_counter() - t0, "traj": traj_right})

    if r_left * r_right > 0:
        raise RuntimeError('Bisection requires residuals with opposite signs on the bracket')

    for _ in range(maxiter - 2):
        mid = 0.5 * (left + right)
        t0 = time.perf_counter()
        r_mid, traj_mid = shooting_residual(mid, integrator_name, h)
        dt = time.perf_counter() - t0
        history.append({"slope": mid, "residual": r_mid, "time": dt, "traj": traj_mid})

        if abs(r_mid) <= tol:
            break
        if r_left * r_mid < 0:
            right, r_right = mid, r_mid
        else:
            left, r_left = mid, r_mid

        if abs(right - left) < slope_tol:
            break

    return history

## Execution + plotting
Run this cell to solve the BVP with the chosen configuration. It prints the final slope and residual, shows the trajectories tried by the root finder, the residual history, and the per-iteration wall time.

In [ ]:
def plot_history(history: List[Dict]):
    if not history:
        raise RuntimeError('No iterations recorded')

    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    cmap = plt.cm.viridis

    for idx, entry in enumerate(history):
        xs, ys, _ = entry['traj']
        axes[0].plot(xs, ys, color=cmap(idx / max(1, len(history) - 1)), label=f"iter {idx}: s={entry['slope']:.4g}")

    axes[0].axhline(alpha, color='k', linestyle='--', linewidth=0.8, label='y(a) target')
    axes[0].axhline(beta, color='gray', linestyle=':', linewidth=0.8, label='y(b) target')
    axes[0].set_title('Trajectories per iteration')
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].legend(loc='best')

    residuals = [abs(h['residual']) for h in history]
    axes[1].semilogy(range(len(history)), residuals, marker='o')
    axes[1].axhline(residual_tol, color='r', linestyle='--', linewidth=0.8, label='tolerance')
    axes[1].set_title('Residual |y(b)-beta|')
    axes[1].set_xlabel('iteration')
    axes[1].set_ylabel('|residual|')
    axes[1].legend(loc='best')

    times_ms = [h['time'] * 1e3 for h in history]
    axes[2].bar(range(len(history)), times_ms)
    axes[2].set_title('Wall time per iteration [ms]')
    axes[2].set_xlabel('iteration')
    axes[2].set_ylabel('ms')

    fig.tight_layout()
    plt.show()

    if true_solution is not None:
        xs_dense = np.linspace(a, b, 400)
        ts = true_solution(xs_dense)
        if ts is not None:
            plt.figure(figsize=(6, 4))
            plt.plot(xs_dense, ts, label='true solution', color='black', linewidth=2)
            xs_best, ys_best, _ = history[-1]['traj']
            plt.plot(xs_best, ys_best, label='numerical', linestyle='--')
            plt.title('True vs numerical (final iterate)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.legend()
            plt.tight_layout()
            plt.show()


if root_method.lower() == 'newton':
    history = run_newton(initial_guess, residual_tol, max_iters, newton_fd_step, step_size, integrator)
elif root_method.lower() == 'secant':
    history = run_secant(initial_guess, secondary_guess, residual_tol, max_iters, step_size, integrator)
elif root_method.lower() == 'bisection':
    history = run_bisection(bracket, residual_tol, max_iters, step_size, integrator)
else:
    raise ValueError('root_method must be one of: newton, secant, bisection')

final = history[-1]
print(f"Final slope: {final['slope']:.12g}")
print(f"Residual y(b)-beta: {final['residual']:.3e}")
print(f"Iterations: {len(history)}")
plot_history(history)